### <span style="color:blue">Quantum States: DumpMachine</span>

In [ ]:
// Run this cell using Ctrl+Enter (⌘+Enter on Mac)
// Then run the next cell to see the output

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

// Let's use some random gates in Task13 and see what effect they have on the |0⟩ state.
// To try another solution, change the gates and re-run both cells
operation Task13 (q : Qubit) : Unit is Adj+Ctl {
    H(q);
    Rx(1.234, q);
}

operation DumpMachineDemo () : Unit {
    // Allocate a qubit - it will start in the |0⟩ state
    use q = Qubit();
    // Apply our operation - it will prepare a state that should be ((1+i)|0⟩ + (1-i)|1⟩) / 2
    Task13(q);
    // Print the current state of the system
    DumpMachine();
    // Return the qubit to the |0⟩ state before releasing
    Reset(q);
}

In [ ]:
%simulate DumpMachineDemo

### <span style="color:blue">Quantum Operations: DumpOperation</span>

In [ ]:
// Run this cell using Ctrl+Enter (⌘+Enter on Mac)
// Then run the next cell to see the output

open Microsoft.Quantum.Diagnostics;

operation SingleQubitGateArrayWrapper (qs : Qubit[]) : Unit is Adj+Ctl {
    Task13(qs[0]);
}

operation DumpOperationDemo () : Unit {
    // Let's look at the matrix representation of the gate we implemented
    DumpOperation(1, SingleQubitGateArrayWrapper);
}

In [ ]:
%simulate DumpOperationDemo

### <span style="color:blue">Circuit visualization: %trace</span>
The algorithm shown here is called Deutsch-Jozsa algorithm; we'll look at this algorithm later in the course.

In [ ]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;

// Phase oracle implementing a constant function f(x) = 0
operation PhaseOracleZero (inputRegister : Qubit[]) : Unit {
    // Do nothing!
}

// Marking oracle implementing a balanced function f(x) = xₖ (the value of k-th bit)
operation MarkingOracleKthBit (inputRegister : Qubit[], target : Qubit, k : Int) : Unit {
    Controlled X([inputRegister[k]], target);
}

operation ApplyMarkingOracleAsPhaseOracle (markingOracle : ((Qubit[], Qubit) => Unit), inputRegister : Qubit[]) : Unit {
    use target = Qubit();
    within {
        // Put the target into the |-⟩ state
        X(target);
        H(target);
    } apply {
        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition will apply a -1 factor to the state
        markingOracle(inputRegister, target);
    }
}

operation IsFunctionConstant (nQubits : Int, phaseOracle : (Qubit[] => Unit)) : Bool {
    mutable isConstant = true;
    use qubits = Qubit[nQubits];
    // Apply the H gates, the oracle and the H gates again
    within {
        ApplyToEachA(H, qubits);
    } apply {
        phaseOracle(qubits);
    }
    // Measure all qubits
    let measurementResults = MultiM(qubits);
    // If any of measurement results are 1, the function is balanced
    for m in measurementResults {
        if (m == One) {
            set isConstant = false;
        }
    }
    return isConstant;
}

function ConstantOrBalanced (value : Bool) : String {
    return value ? "constant" | "balanced";
}

operation RunDeutschJozsaAlgorithm (nQubit : Int) : Unit {
    // for balanced function
    let phaseOracleKthBit = ApplyMarkingOracleAsPhaseOracle(MarkingOracleKthBit(_, _, 1), _);
    let isKthBitConstant = IsFunctionConstant(nQubit, phaseOracleKthBit);
    Message($"f(x) = xk classified as {ConstantOrBalanced(isKthBitConstant)}");
}

In [ ]:
%simulate RunDeutschJozsaAlgorithm nQubit=2

In [ ]:
%trace RunDeutschJozsaAlgorithm nQubit=2

In [ ]:
%debug RunDeutschJozsaAlgorithm nQubit=2

In [ ]:
%estimate RunDeutschJozsaAlgorithm nQubit=3

In [ ]:
%version

In [ ]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Synthesis;
open Microsoft.Quantum.Diagnostics;

operation ApplyH (register : LittleEndian) : Unit is Adj + Ctl {
    let matrix = [[Complex(Sqrt(0.5), 0.0), Complex(Sqrt(0.5), 0.0)],
                  [Complex(Sqrt(0.5), 0.0), Complex(-Sqrt(0.5), 0.0)]];
    ApplyUnitary(matrix, register);
}

operation ShowDump () : Unit {
    use q = Qubit();
    X(q);
    ApplyH(LittleEndian([q]));
    DumpMachine();
}

In [ ]:
%simulate ShowDump